# Lab 15: Observability for AI Agents and Workflows

> ⚠️ **Beta**: This notebook is still in development.  

This lab demonstrates how to add comprehensive **tracing and observability** to:

1. **Simple Agents** - Hello World style single-agent patterns (from Lab 1b)
2. **Multi-Agent Workflows** - Complex orchestration with critique loops (from Lab 12)

## What You'll Learn

| Concept | Description |
|---------|-------------|
| OpenTelemetry Setup | Configure distributed tracing for AI services |
| Azure Monitor Integration | Send traces to Application Insights |
| Agent Instrumentation | Automatic tracing for Foundry SDK calls |
| Custom Spans | Manual span creation for business logic |
| Debug Event Emission | Real-time workflow progress tracking |

## Prerequisites

Complete these labs first:
- **Lab 1a** - Landing Zone deployment (APIM Gateway)
- **Lab 1b** - Project Spoke deployment (optional, for Foundry agents)

## Step 1: Install Dependencies

In [ ]:
# Core tracing dependencies
%pip install -q opentelemetry-sdk opentelemetry-api

# Azure Monitor exporter (for Application Insights)
%pip install -q azure-monitor-opentelemetry

# Optional: OTLP exporters for local tracing (Jaeger/Zipkin)
%pip install -q opentelemetry-exporter-otlp-proto-grpc opentelemetry-exporter-otlp-proto-http

# Agent Framework
%pip install -q agent-framework --pre

# Azure AI Projects SDK with telemetry support
%pip install -q azure-ai-projects==2.0.0b2 azure-identity

## Step 2: Load Configuration

In [ ]:
import os
import sys
from pathlib import Path

# Add this directory to path for local imports
sys.path.insert(0, str(Path.cwd()))

# Load .env from parent directory
env_path = Path("../.env")
if env_path.exists():
    for line in env_path.read_text().splitlines():
        if '=' in line and not line.startswith('#'):
            key, value = line.split('=', 1)
            os.environ[key.strip()] = value.strip()

# Landing Zone settings (from Lab 1a)
APIM_URL = os.environ.get("APIM_URL", "")
APIM_KEY = os.environ.get("APIM_KEY", "")
MODEL_NAME = os.environ.get("MODEL_NAME", "gpt-4.1-mini")

# Spoke settings (from Lab 1b) - optional for Foundry agents
SPOKE_ENDPOINT = os.environ.get("SPOKE_ENDPOINT", "")
SPOKE_PROJECT = os.environ.get("SPOKE_PROJECT", "")

# Application Insights (optional - for production tracing)
APP_INSIGHTS_CONN = os.environ.get("APPLICATIONINSIGHTS_CONNECTION_STRING", "")

print("📋 Configuration:")
print(f"   APIM URL:         {APIM_URL[:50]}..." if APIM_URL else "   ❌ APIM_URL not set")
print(f"   Model:            {MODEL_NAME}")
print(f"   App Insights:     {'✅ Configured' if APP_INSIGHTS_CONN else '⚠️ Not configured (will use console)'}")

## Step 3: Initialize Tracing

This is the **critical step** - tracing must be initialized **before** creating any agents or making LLM calls.

In [15]:
from tracing import setup_tracing, is_tracing_enabled, get_trace_count, reset_trace_count

# Enable tracing (will use console if no App Insights configured)
os.environ["TRACING_ENABLED"] = "true"

# Initialize tracing
tracing_active = setup_tracing(
    service_name="foundry-observability-lab",
    enable_content_recording=True  # Record prompts and completions in traces
)

print(f"\n🔍 Tracing Status: {'✅ ACTIVE' if tracing_active else '❌ INACTIVE'}")
print(f"   Content Recording: {'✅ Enabled' if tracing_active else 'N/A'}")


🔍 Tracing Status: ✅ ACTIVE
   Content Recording: ✅ Enabled


---

# Part 1: Simple Agent with Tracing

Let's start with a simple "Hello World" agent (similar to Lab 1b) and add observability.

## 1.1 Create Agent with APIM Gateway

In [16]:
from agent_framework.openai import OpenAIChatClient
from agent_framework import ChatMessage, Role

# Create chat client using APIM Gateway
# The tracing is automatically instrumented by the Azure SDK settings
chat_client = OpenAIChatClient(
    model_id=MODEL_NAME,
    api_key="placeholder",  # APIM uses header auth
    base_url=f"{APIM_URL}/deployments/{MODEL_NAME}",
    default_headers={"api-key": APIM_KEY},
)

# Create a simple agent
hello_agent = chat_client.create_agent(
    name="HelloWorldAgent",
    instructions="""You are a friendly assistant that demonstrates observability.
When responding, always mention that you're being traced by OpenTelemetry.
Keep responses brief and cheerful.""",
    model=MODEL_NAME,
)

print(f"✅ Created agent: {hello_agent.name}")
print(f"   Model: {MODEL_NAME}")
print(f"   Tracing: {'Enabled' if is_tracing_enabled() else 'Disabled'}")

✅ Created agent: HelloWorldAgent
   Model: gpt-4.1-mini
   Tracing: Enabled


## 1.2 Invoke Agent (with Automatic Tracing)

When we call the agent, OpenTelemetry automatically captures:
- Request/response spans
- Token usage metrics
- Latency information
- Prompt and completion content (if enabled)

In [ ]:
import time

# Reset trace count before our test
reset_trace_count()
start_traces = get_trace_count()

# Call the agent
query = "Hello! Can you tell me about observability in AI systems?"
print(f"🧑 User: {query}")
print("\n⏳ Calling agent (with tracing)...\n")

start_time = time.time()
response = await hello_agent.run(
    [ChatMessage(role=Role.USER, text=query)]
)
duration_ms = (time.time() - start_time) * 1000

# Display response
print(f"🤖 Agent: {response.text}")
print(f"\n📊 Metrics:")
print(f"   Duration: {duration_ms:.0f}ms")
print(f"   Traces recorded: {get_trace_count() - start_traces}")

## 1.3 Add Custom Spans with Decorator

For business logic that wraps agent calls, use the `@with_agent_telemetry` decorator:

In [ ]:
from maf_wrapper import with_agent_telemetry

@with_agent_telemetry("SummarizationAgent", "Summarize user input")
async def summarize_text(text: str) -> str:
    """
    This function is automatically traced with:
    - A parent span named 'SummarizationAgent.summarize_text'
    - Input text captured in span attributes
    - Output captured in span attributes
    - Timing and error tracking
    """
    response = await hello_agent.run([
        ChatMessage(
            role=Role.USER,
            text=f"Summarize this in one sentence: {text}"
        )
    ])
    return response.text

# Test the decorated function
long_text = """
OpenTelemetry is an observability framework for cloud-native software.
It provides APIs, libraries, and tools to collect distributed traces,
metrics, and logs from your applications. The project is a merger of
OpenTracing and OpenCensus projects.
"""

print("📝 Input text (truncated):")
print(f"   {long_text.strip()[:100]}...")
print("\n⏳ Calling decorated function...\n")

summary = await summarize_text(long_text)
print(f"📋 Summary: {summary}")

## 1.4 Manual Span Creation

For fine-grained control, create spans manually:

In [ ]:
from tracing import get_tracer
from opentelemetry import trace as otel_trace

tracer = get_tracer("my_custom_logic")

async def process_with_custom_spans(user_input: str) -> dict:
    """
    Demonstrates manual span creation for complex business logic.
    """
    # Parent span for the entire operation
    with tracer.start_as_current_span(
        "process_user_request",
        kind=otel_trace.SpanKind.INTERNAL
    ) as parent_span:
        parent_span.set_attribute("user.input_length", len(user_input))
        
        # Child span for preprocessing
        with tracer.start_as_current_span("preprocess") as prep_span:
            cleaned_input = user_input.strip().lower()
            prep_span.set_attribute("preprocessing.result_length", len(cleaned_input))
        
        # Child span for agent call
        with tracer.start_as_current_span(
            "call_agent",
            kind=otel_trace.SpanKind.CLIENT
        ) as agent_span:
            agent_span.set_attribute("agent.name", "HelloWorldAgent")
            agent_span.set_attribute("gen_ai.prompt", cleaned_input[:200])
            
            response = await hello_agent.run([
                ChatMessage(role=Role.USER, text=cleaned_input)
            ])
            
            agent_span.set_attribute("gen_ai.completion", response.text[:200])
        
        # Child span for postprocessing
        with tracer.start_as_current_span("postprocess") as post_span:
            result = {
                "input": user_input,
                "response": response.text,
                "word_count": len(response.text.split())
            }
            post_span.set_attribute("postprocessing.word_count", result["word_count"])
        
        parent_span.set_status(otel_trace.StatusCode.OK)
        return result

# Test manual spans
result = await process_with_custom_spans("What is distributed tracing?")
print(f"✅ Processed request with {result['word_count']} words in response")
print(f"📊 Total traces recorded: {get_trace_count()}")

---

# Part 2: Multi-Agent Workflow with Tracing

Now let's add observability to a more complex multi-agent workflow (similar to Lab 12).

## 2.1 Create Debug Event Emitter

For real-time workflow monitoring, we use a debug event emitter:

In [20]:
from debug_events import DebugEventEmitter, create_debug_emitter
from IPython.display import display, HTML
import json

# Create an emitter that displays events in real-time
def display_event(event: dict):
    """Display workflow events in the notebook."""
    event_type = event.get("type", "unknown")
    
    # Color-code by event type
    colors = {
        "workflow_started": "#4CAF50",
        "workflow_completed": "#4CAF50",
        "phase_started": "#2196F3",
        "llm_call_started": "#FF9800",
        "llm_call_completed": "#8BC34A",
        "llm_call_failed": "#f44336",
        "search_started": "#9C27B0",
        "search_completed": "#9C27B0",
    }
    color = colors.get(event_type, "#666")
    
    # Format event for display
    summary = ""
    if "agent" in event:
        summary = f"Agent: {event['agent']}"
    if "duration_ms" in event:
        summary += f" ({event['duration_ms']}ms)"
    if "description" in event:
        summary = event["description"]
    
    print(f"  📍 [{event_type}] {summary}")

# Create emitter with our display callback
emitter = create_debug_emitter(callback=display_event)
print("✅ Debug event emitter created")

✅ Debug event emitter created


## 2.2 Workflow Tracer Context Manager

The `WorkflowTracer` provides automatic span management for multi-step workflows:

In [21]:
from maf_wrapper import WorkflowTracer
from pydantic import BaseModel, Field
from typing import List

# Define some models for structured output
class SlideOutline(BaseModel):
    position: int
    topic: str
    search_keywords: List[str] = Field(default_factory=list)

class PresentationOutline(BaseModel):
    title: str
    narrative: str
    slides: List[SlideOutline]

# Create a traced workflow
async def build_presentation_workflow(topic: str) -> dict:
    """
    Demonstrates a multi-step workflow with comprehensive tracing.
    
    This simulates the pattern from Lab 12 where multiple agents
    collaborate to build a presentation.
    """
    results = {"topic": topic, "slides": []}
    
    async with WorkflowTracer(
        "presentation_builder",
        total_steps=3,
        event_callback=display_event
    ) as wt:
        
        # Step 1: Planning
        wt.start_step("planning", "Generate presentation outline")
        
        emitter.llm_call_started(
            agent="PlannerAgent",
            task="Generate outline",
            prompt_preview=f"Create outline for: {topic}",
            response_format="PresentationOutline"
        )
        
        # Call the planner (using our simple agent for demo)
        import time
        start = time.time()
        
        plan_response = await hello_agent.run([
            ChatMessage(
                role=Role.USER,
                text=f"Create a 3-slide outline for a presentation about: {topic}. List each slide topic briefly."
            )
        ])
        
        duration = int((time.time() - start) * 1000)
        emitter.llm_call_completed(
            agent="PlannerAgent",
            duration_ms=duration,
            response_preview=plan_response.text[:200]
        )
        
        results["outline"] = plan_response.text
        wt.complete_step(success=True, details={"outline_length": len(plan_response.text)})
        
        # Step 2: Search (simulated)
        wt.start_step("search", "Find relevant images")
        
        emitter.search_started(query=f"{topic} images")
        await asyncio.sleep(0.5)  # Simulate search
        emitter.search_completed(
            query=f"{topic} images",
            result_count=5,
            duration_ms=500
        )
        
        wt.complete_step(success=True, details={"images_found": 5})
        
        # Step 3: Assembly
        wt.start_step("assembly", "Assemble final presentation")
        
        # Simulate selection with review
        for i in range(3):
            emitter.selection_made(
                position=i + 1,
                selected_id=f"image_{i}",
                reason=f"Best match for slide {i + 1}"
            )
            emitter.review_completed(
                position=i + 1,
                approved=True,
                feedback="Image matches the topic well"
            )
            results["slides"].append({"position": i + 1, "image_id": f"image_{i}"})
        
        wt.complete_step(success=True, details={"slides_assembled": 3})
    
    return results

import asyncio
print("✅ Workflow function defined")

✅ Workflow function defined


## 2.3 Run the Traced Workflow

In [ ]:
# Reset trace count
reset_trace_count()

print("🚀 Starting traced workflow...")
print("="*50)

result = await build_presentation_workflow("OpenTelemetry and Distributed Tracing")

print("="*50)
print(f"\n✅ Workflow completed!")
print(f"   Slides created: {len(result['slides'])}")
print(f"   Traces recorded: {get_trace_count()}")

print(f"\n📋 Outline preview:")
print(f"   {result['outline'][:300]}...")

## 2.4 View All Captured Events

In [23]:
print(f"📊 All captured events ({len(emitter.events)} total):")
print("="*60)

for event in emitter.events:
    event_type = event.get("type", "unknown")
    timestamp = event.get("timestamp", 0)
    
    # Format event details
    details = {k: v for k, v in event.items() if k not in ["type", "timestamp"]}
    details_str = json.dumps(details, default=str)[:100]
    
    print(f"  [{event_type:20}] {details_str}")

📊 All captured events (10 total):
  [llm_call_started    ] {"agent": "PlannerAgent", "task": "Generate outline", "prompt_preview": "Create outline for: OpenTel
  [llm_call_completed  ] {"agent": "PlannerAgent", "duration_ms": 1580, "response_preview": "Sure! Here's a cheerful 3-slide 
  [search_started      ] {"query": "OpenTelemetry and Distributed Tracing images"}
  [search_completed    ] {"query": "OpenTelemetry and Distributed Tracing images", "result_count": 5, "duration_ms": 500, "re
  [selection_made      ] {"position": 1, "selected_id": "image_0", "reason": "Best match for slide 1"}
  [review_completed    ] {"position": 1, "approved": true, "feedback": "Image matches the topic well"}
  [selection_made      ] {"position": 2, "selected_id": "image_1", "reason": "Best match for slide 2"}
  [review_completed    ] {"position": 2, "approved": true, "feedback": "Image matches the topic well"}
  [selection_made      ] {"position": 3, "selected_id": "image_2", "reason": "Best match for 

---

# Summary

## What We Learned

| Concept | Implementation |
|---------|----------------|
| **Tracing Setup** | `setup_tracing()` - Call once at startup |
| **Auto-Instrumentation** | Azure SDK automatically traces all LLM calls |
| **Custom Spans** | `get_tracer()` + context managers |
| **Decorated Functions** | `@with_agent_telemetry()` decorator |
| **Workflow Tracing** | `WorkflowTracer` context manager |
| **Debug Events** | `DebugEventEmitter` for real-time monitoring |

## Key Files in This Lab

| File | Purpose |
|------|--------|
| `tracing.py` | OpenTelemetry setup and configuration |
| `maf_wrapper.py` | Agent telemetry decorators and workflow tracer |
| `debug_events.py` | Event emission for real-time monitoring |

## Next Steps

1. **Deploy Application Insights** for production monitoring
2. **Create dashboards** for agent performance metrics
3. **Set up alerts** for error rates and latency spikes
4. **Add custom metrics** for business KPIs

In [24]:
# Final trace count
print(f"\n📊 Final Statistics:")
print(f"   Total traces recorded: {get_trace_count()}")
print(f"   Debug events captured: {len(emitter.events)}")
print(f"   Tracing active: {is_tracing_enabled()}")


📊 Final Statistics:
   Total traces recorded: 1
   Debug events captured: 10
   Tracing active: True
